<a href="https://colab.research.google.com/github/SomdeepAcharyya/Recommender-Systems/blob/main/Model_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# source to be clustered with dbscan/kmeans
# target instances to be merged with clusters from source
# joint training and embedding on individual subsets
# prediction on target sets

In [ ]:
import numpy as np
import scipy.sparse as sp
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error 
import scipy.spatial as spt
import statistics
import math
from scipy import spatial
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# data collection

In [ ]:
# Tripadvisor review Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/tripadvisor_reviews_with_country.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  tr = pd.read_csv(infile)
tr = tr.rename(columns={"username":"userId", "taObject":"itemId"})
arr = tr[['open', 'cons', 'extra', 'agree', 'neuro', 'userId', 'itemId', 'rating']]

In [ ]:
# amazon review dataset movies json
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_magazine_filtered.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  az = pd.read_csv(infile, nrows=10000)
#am = am.rename(columns={"reviewerID":"userId", "asin":"itemId", "overall":"rating"})
az.shape

(10000, 6)

In [ ]:
arr = tr.processed_text
source_texts = np.array(tr.processed_text)
target_texts = np.array(az.processed_text)

# Clustering of text

In [ ]:
def cluster_text(X):
    import matplotlib.pyplot as plt
    from sklearn.cluster import KMeans
    Sum_of_squared_distances = []
    K = range(2,10)
    for k in K:
       km = KMeans(n_clusters=k, max_iter=200, n_init=10)
       km = km.fit(X)
       Sum_of_squared_distances.append(km.inertia_)
    plt.plot(K, Sum_of_squared_distances, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Sum_of_squared_distances')
    plt.title('Elbow Method For Optimal k')
    plt.show()

    print('How many clusters do you want to use?')
    true_k = int(input())
    model = KMeans(n_clusters=true_k, init='k-means++', max_iter=200, n_init=10)
    model.fit(X)

    labels=model.labels_
    clusters=pd.DataFrame(list(zip(X,labels)),columns=['title','cluster'])
    #print(clusters.sort_values(by=['cluster']))

    for i in range(true_k):
        print(clusters[clusters['cluster'] == i])
        
    return clusters

In [ ]:
s = arr.fillna(" ")
t = az.processed_text.fillna(" ")

In [ ]:
True in np.array(s.isna())

False

In [ ]:
True in np.array(t.isna())

False

In [ ]:
vectorizer = CountVectorizer(max_features = 1000) 
vectorizer.fit(s) 
source_embeddings = vectorizer.transform(s)
vectorizer.fit(t) 
target_embeddings = vectorizer.transform(t)

In [ ]:
c = cluster_text(source_embeddings)

In [ ]:
set_source = []
true_k = len(c.cluster.value_counts())
for k in range(0, true_k):
  set_source.append(np.array(c[c['cluster'] == k].title))

In [ ]:
c.to_csv(r'/content/drive/MyDrive/Per_CD_RS/magzc.csv') 

In [ ]:
# sim v1
li = []
temp = 100
li_total = []
counter = 0
for i in target_embeddings:
  if counter % 200 == 0:
    print(counter)
  for j in range(len(set_source)):
    sim = []
    for k in set_source[j]:
      val1 = math.exp(1 - spatial.distance.cosine(i.toarray(), k.toarray()) / temp)
      sim.append(val1)
    denom = sum(sim)  
    for k in sim: 
      val2 = -1 * math.log(k) / denom
      li.append(val2)
    li_total.append(sum(li) / len(li))
    li = []
  counter = counter + 1

0


In [ ]:
# sim v2
li = []
denom = []
temp = 100
li_total = []
for i in target_embeddings:
  for j in range(len(set_source)):
    sim = []
    for k in set_source[j]:
      val = math.exp(1 - spatial.distance.cosine(i.toarray(), k.toarray()) / temp)
      sim.append(val)
    denom.append(sum(sim))
  for j in range(len(set_source)):
    for k in set_source[j]:
      val = -1 * math.log(math.exp(1 - spatial.distance.cosine(i.toarray(), k.toarray()) / temp) / denom[j])
      li.append(val)
    li_total.append(sum(li) / len(li))
    li = []

In [ ]:
# sim v3
li = []
li_total = []
counter = 0
for i in target_embeddings:
  if counter % 200 == 0:
    print(counter)
  for j in range(len(set_source)):
    sim = []
    for k in set_source[j]:
      val1 = 1 - spatial.distance.cosine(i.toarray(), k.toarray())
      sim.append(val1)
    denom = sum(sim)  
    for k in sim: 
      val2 = -1 * math.log(k+1) / denom
      li.append(val2)
    li_total.append(sum(li) / len(li))
    li = []
  counter = counter + 1 

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000


In [ ]:
 li_all = np.reshape(li_total, (target_embeddings.shape[0], int(len(li_total)/target_embeddings.shape[0])))

In [ ]:
li_total

In [ ]:
index_of_cluster = []
for i in li_all:
  i = list(i) 
  index = i.index(min(i))
  index_of_cluster.append(index)

In [ ]:
source_clusters = c
target_clusters = pd.DataFrame(list(zip(target_embeddings,index_of_cluster, az.rating, az.userId)),columns=['title','cluster', 'rating', 'userId'])

In [ ]:
source_clusters['text'] = source_texts
target_clusters['text'] = target_texts

In [ ]:
target_clusters.cluster.value_counts()

In [ ]:
target_clusters.to_csv(r'/content/drive/MyDrive/Per_CD_RS/magz_clusters1.csv') 

# BERT

In [ ]:
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
from transformers import logging
logging.set_verbosity_error()

In [ ]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.overwrite_output_dir = True

In [ ]:
pers_s = np.array(tr[['open', 'cons', 'extra', 'agree', 'neuro']])
source_clusters['userId'] = np.array(tr['userId'])
new_s = source_clusters.merge(tr[['userId', 'open', 'cons', 'extra', 'agree', 'neuro']])

In [ ]:
target_clusters.cluster.value_counts()

NameError: ignored

In [ ]:
target_output = pd.DataFrame()
for i in range(0, true_k):
  x = new_s[new_s['cluster']==i]
  y = target_clusters[target_clusters['cluster']==i]
  z = pd.concat([x,y])
  lx = len(x)
  ly = len(y)
  s = ['cons']
  z = z.fillna(0)
  t = pd.DataFrame()
  t['userId'] = z['userId'][lx:len(z)]
  t['open'] = 0.0
  t['cons'] = 0.0
  t['extra'] = 0.0
  t['agree'] = 0.0
  t['neuro'] = 0.0
  for j in range(len(s)):
    try:
      sample = z[['text', 'open', 'cons', 'extra', 'agree', 'neuro']]
      a = sample[0:lx]
      b = sample[lx:len(sample)]
      if len(b) > 0:
        model = ClassificationModel("roberta", "roberta-base", num_labels=1, args=model_args, use_cuda=False)
        print("1")
        model.train_model(a)
        print("2")
        result, model_outputs, wrong_predictions = model.eval_model(b)
        print("3")
        print(result, model_outputs, wrong_predictions)
        print("4")
        t[s[j]] = model_outputs
      else:
        print(i, j, "b empty")
    except:
          print("error")
  target_output = pd.concat([target_output, t])

In [ ]:
target_output.to_csv(r'/content/drive/MyDrive/Per_CD_RS/magztest.csv') 